In [110]:
import awswrangler as wr
import pandas as pd

# Athena 쿼리 실행 → Pandas DataFrame
df = wr.athena.read_sql_query(
    sql="""
        SELECT a.test_id, a.test_name, a.essential_info,
               b.scale_code, b.scale_name, b.parent_scale_id, b.level, b.version
        FROM test_info a
        INNER JOIN scale_tree b
          ON a.test_id = b.test_id
         AND a.version = b.version
        WHERE a.version = 'V 1.0'
    """,
    database="saas_proto",
    ctas_approach=False,                           # 기본은 CTAS, False로 두면 그냥 SELECT 실행
    s3_output="s3://saas-inpsyt/athena-query-results/"    # 쿼리 결과 저장될 S3 경로
)

print(df.head())

        test_id   test_name essential_info scale_code scale_name  \
0  GOLDEN_CO_SG  골든 성격유형 검사     {sex, age}        t02        분석적   
1  GOLDEN_CO_SG  골든 성격유형 검사     {sex, age}        t01        논리적   
2  GOLDEN_CO_SG  골든 성격유형 검사     {sex, age}          T         사고   
3  GOLDEN_CO_SG  골든 성격유형 검사     {sex, age}        a02        친밀한   
4  GOLDEN_CO_SG  골든 성격유형 검사     {sex, age}        a01      말이 많은   

  parent_scale_id  level version  
0               T      3   V 1.0  
1               T      3   V 1.0  
2           TOTAL      2   V 1.0  
3               E      3   V 1.0  
4               E      3   V 1.0  


In [15]:
grouped_testid_df = df.groupby(['test_id'])
tree_dict = {}

for test_id, group in grouped_testid_df:
    print(f"Processing test_id: {test_id}")
    tree_dict[test_id] = {} # 검사 명 딕셔너리
    tree_dict[test_id] = {'test_name': group['test_name'].iloc[0], 'essential_info': group['essential_info'].iloc[0], 'version': group['version'].iloc[0], 'scale_tree': {}}
    group = group.sort_values(by=['level', 'scale_code']) # level, scale_code 기준 정렬
    print(group)
    for idx, row in group.iterrows(): # 인덱스 순서대로 하나씩 행 순환
        # level 중 제일 낮은 행 선택 -> 최상위 척도 (부모)
        if row['level'] == group['level'].min() and row['version'] == tree_dict[test_id]['version']:
            tree_dict[test_id][row['scale_code']] = {'scale_code': row['scale_code'], 'scale_name': row['scale_name'], 'children': {}}
        elif row['level'] > group['level'].min() and row['level'] < group['level'].max() and row['version'] == tree_dict[test_id]['version'] and row['parent_scale_id'] == tree_dict[test_id][row['parent_scale_id']]['scale_code']:
            tree_dict[test_id][row['parent_scale_id']]['children'][row['scale_code']] = {'scale_code': row['scale_code'], 'scale_name': row['scale_name'], 'children': {}}
        elif row['level'] == group['level'].max() and row['version'] == tree_dict[test_id]['version'] and row['parent_scale_id'] == tree_dict[test_id][row['parent_scale_id']]['scale_code']:

#! 이렇게 하니까 조건문이 레벨별로 늘어나네... 

Processing test_id: ('ADHD_CO_SG',)
       test_id test_name essential_info scale_code scale_name parent_scale_id  \
12  ADHD_CO_SG    주의력 검사          {age}          A        주의력            <NA>   
11  ADHD_CO_SG    주의력 검사          {age}          B       과잉행동            <NA>   

    level version  
12      1   V 1.0  
11      1   V 1.0  
Processing test_id: ('GOLDEN_CO_SG',)
        test_id   test_name essential_info scale_code scale_name  \
6  GOLDEN_CO_SG  골든 성격유형 검사     {sex, age}      TOTAL         종합   
5  GOLDEN_CO_SG  골든 성격유형 검사     {sex, age}          E         외향   
2  GOLDEN_CO_SG  골든 성격유형 검사     {sex, age}          T         사고   
4  GOLDEN_CO_SG  골든 성격유형 검사     {sex, age}        a01      말이 많은   
3  GOLDEN_CO_SG  골든 성격유형 검사     {sex, age}        a02        친밀한   
1  GOLDEN_CO_SG  골든 성격유형 검사     {sex, age}        t01        논리적   
0  GOLDEN_CO_SG  골든 성격유형 검사     {sex, age}        t02        분석적   

  parent_scale_id  level version  
6            <NA>      1   V 1.0  
5      

In [111]:
# 재귀 헬퍼 함수
def find_node(tree, parent_code):
    """트리(dict) 안에서 parent_code에 해당하는 노드를 찾아 반환"""
    if parent_code in tree:  # 현재 depth에서 찾은 경우
        return tree[parent_code]
    
    # children 재귀 탐색
    for node in tree.values():
        if "children" in node:
            found = find_node(node["children"], parent_code)
            if found:
                return found
    return None

In [ ]:
# 재귀 헬퍼 함수 (디버깅용)
def find_node(tree, parent_code, depth=0):
    indent = "  " * depth  # 보기 좋게 들여쓰기
    print(f"{indent}find_node 호출: parent_code={parent_code}, 현재 keys={list(tree.keys())}")
    
    if parent_code in tree:
        print(f"{indent}👉 현재 depth에서 {parent_code} 발견! 반환")
        return tree[parent_code]
    
    for key, node in tree.items():
        if "children" in node:
            print(f"{indent}{key}의 children 탐색 진입")
            found = find_node(node["children"], parent_code, depth+1)
            if found:
                return found
    
    print(f"{indent}❌ {parent_code} 여기서는 못 찾음")
    return None

In [116]:
grouped_testid_df = df.groupby('test_id')
tree_dict = {}

for test_id, group in grouped_testid_df:
    print(f"Processing test_id: {test_id}")
    tree_dict[test_id] = {
        'test_name': group['test_name'].iloc[0],
        'essential_info': group['essential_info'].iloc[0],
        'version': group['version'].iloc[0],
        'scale_tree': {}
    }

    # level, scale_code 기준 정렬
    group = group.sort_values(by=['level', 'scale_code'])
    print(group)
    max_level = group['level'].max()
    min_level = group['level'].min()

    for idx, row in group.iterrows():
        node = {
            "scale_name": row['scale_name'],
            "children": {}
        }

        if pd.isna(row['parent_scale_id']):  # 최상위 척도
            tree_dict[test_id]['scale_tree'][row['scale_code']] = node
        else:
            # parent 위치 찾아서 children에 추가
            parent_node = find_node(tree_dict[test_id]['scale_tree'], row['parent_scale_id'])
            if parent_node:
                parent_node['children'][row['scale_code']] = node

Processing test_id: ADHD_CO_SG
       test_id test_name essential_info scale_code scale_name parent_scale_id  \
12  ADHD_CO_SG    주의력 검사          {age}          A        주의력            <NA>   
11  ADHD_CO_SG    주의력 검사          {age}          B       과잉행동            <NA>   

    level version  
12      1   V 1.0  
11      1   V 1.0  
Processing test_id: GOLDEN_CO_SG
        test_id   test_name essential_info scale_code scale_name  \
6  GOLDEN_CO_SG  골든 성격유형 검사     {sex, age}      TOTAL         종합   
5  GOLDEN_CO_SG  골든 성격유형 검사     {sex, age}          E         외향   
2  GOLDEN_CO_SG  골든 성격유형 검사     {sex, age}          T         사고   
4  GOLDEN_CO_SG  골든 성격유형 검사     {sex, age}        a01      말이 많은   
3  GOLDEN_CO_SG  골든 성격유형 검사     {sex, age}        a02        친밀한   
1  GOLDEN_CO_SG  골든 성격유형 검사     {sex, age}        t01        논리적   
0  GOLDEN_CO_SG  골든 성격유형 검사     {sex, age}        t02        분석적   

  parent_scale_id  level version  
6            <NA>      1   V 1.0  
5           TOTAL

In [117]:
tree_dict

{'ADHD_CO_SG': {'test_name': '주의력 검사',
  'essential_info': '{age}',
  'version': 'V 1.0',
  'scale_tree': {'A': {'scale_name': '주의력', 'children': {}},
   'B': {'scale_name': '과잉행동', 'children': {}}}},
 'GOLDEN_CO_SG': {'test_name': '골든 성격유형 검사',
  'essential_info': '{sex, age}',
  'version': 'V 1.0',
  'scale_tree': {'TOTAL': {'scale_name': '종합',
    'children': {'E': {'scale_name': '외향',
      'children': {'a01': {'scale_name': '말이 많은', 'children': {}},
       'a02': {'scale_name': '친밀한', 'children': {}}}},
     'T': {'scale_name': '사고',
      'children': {'t01': {'scale_name': '논리적', 'children': {}},
       't02': {'scale_name': '분석적', 'children': {}}}}}}}},
 'IESS_CO_SG': {'test_name': '통합 스트레스 검사',
  'essential_info': '{age, position, role}',
  'version': 'V 1.0',
  'scale_tree': {'A': {'scale_name': '비관주의', 'children': {}},
   'B': {'scale_name': '수면문제', 'children': {}},
   'C': {'scale_name': '우울', 'children': {}}}},
 'OHSS_CO_SG': {'test_name': '직무스트레스 검사',
  'essential_info': '

In [56]:
test_dict = {}
test_dict['GOLDEN_CO_SG'] = {
    'test_name': '골든 성격유형 검사',
    'essential_info': '{sex, age}',
    'version': 'V 1.0',
    'scale_tree': {}
}

In [58]:
test_dict

{'GOLDEN_CO_SG': {'test_name': '골든 성격유형 검사',
  'essential_info': '{sex, age}',
  'version': 'V 1.0',
  'scale_tree': {}}}

In [51]:
golden_df = grouped_testid_df.get_group('GOLDEN_CO_SG')
golden_df = golden_df.sort_values(by=['level', 'scale_code'], ascending=True)
golden_df

/var/folders/bq/z2chppb96j70s_2s04l9nlvc0000gn/T/ipykernel_11890/1565822685.py:1: FutureWarning: When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.
  golden_df = grouped_testid_df.get_group('GOLDEN_CO_SG')


,test_id,test_name,essential_info,scale_code,scale_name,parent_scale_id,level,version
6,GOLDEN_CO_SG,골든 성격유형 검사,"{sex, age}",TOTAL,종합,<NA>,1,V 1.0
5,GOLDEN_CO_SG,골든 성격유형 검사,"{sex, age}",E,외향,TOTAL,2,V 1.0
2,GOLDEN_CO_SG,골든 성격유형 검사,"{sex, age}",T,사고,TOTAL,2,V 1.0
4,GOLDEN_CO_SG,골든 성격유형 검사,"{sex, age}",a01,말이 많은,E,3,V 1.0
3,GOLDEN_CO_SG,골든 성격유형 검사,"{sex, age}",a02,친밀한,E,3,V 1.0
1,GOLDEN_CO_SG,골든 성격유형 검사,"{sex, age}",t01,논리적,T,3,V 1.0
0,GOLDEN_CO_SG,골든 성격유형 검사,"{sex, age}",t02,분석적,T,3,V 1.0


In [54]:
golden_df_row_1 = golden_df.iloc[0]
golden_df_row_1

test_id            GOLDEN_CO_SG
test_name            골든 성격유형 검사
essential_info       {sex, age}
scale_code                TOTAL
scale_name                   종합
parent_scale_id            <NA>
level                         1
version                   V 1.0
Name: 6, dtype: object

In [60]:
node = {
    "scale_name": golden_df_row_1['scale_name'],
    "children": {}
}
print(node)

{'scale_name': '종합', 'children': {}}


In [66]:
if pd.isna(golden_df_row_1['parent_scale_id']):  # 최상위 척도
    test_dict['GOLDEN_CO_SG']['scale_tree'][golden_df_row_1['scale_code']] = node
print(test_dict)

{'GOLDEN_CO_SG': {'test_name': '골든 성격유형 검사', 'essential_info': '{sex, age}', 'version': 'V 1.0', 'scale_tree': {'TOTAL': {'scale_name': '종합', 'children': {}}}}}


In [67]:
golden_df_row_2 = golden_df.iloc[1]
golden_df_row_2

test_id            GOLDEN_CO_SG
test_name            골든 성격유형 검사
essential_info       {sex, age}
scale_code                    E
scale_name                   외향
parent_scale_id           TOTAL
level                         2
version                   V 1.0
Name: 5, dtype: object

In [73]:
node = {
    "scale_name": golden_df_row_2['scale_name'],
    "children": {}
}

In [69]:
if pd.isna(golden_df_row_2['parent_scale_id']):  # 최상위 척도
    test_dict['GOLDEN_CO_SG']['scale_tree'][golden_df_row_2['scale_code']] = node
print(test_dict)
print(test_dict['GOLDEN_CO_SG']['scale_tree'])

{'GOLDEN_CO_SG': {'test_name': '골든 성격유형 검사', 'essential_info': '{sex, age}', 'version': 'V 1.0', 'scale_tree': {'TOTAL': {'scale_name': '종합', 'children': {}}}}}
{'TOTAL': {'scale_name': '종합', 'children': {}}}


In [72]:
parent_node = find_node(test_dict['GOLDEN_CO_SG']['scale_tree'], golden_df_row_2['parent_scale_id'])
print(parent_node)

현재 depth에서 찾음
{'scale_name': '종합', 'children': {}}


In [74]:
parent_node['children'][golden_df_row_2['scale_code']] = node
print(test_dict)

{'GOLDEN_CO_SG': {'test_name': '골든 성격유형 검사', 'essential_info': '{sex, age}', 'version': 'V 1.0', 'scale_tree': {'TOTAL': {'scale_name': '종합', 'children': {'E': {'scale_name': '외향', 'children': {}}}}}}}


In [75]:
golden_df_row_3 = golden_df.iloc[2]
golden_df_row_3

test_id            GOLDEN_CO_SG
test_name            골든 성격유형 검사
essential_info       {sex, age}
scale_code                    T
scale_name                   사고
parent_scale_id           TOTAL
level                         2
version                   V 1.0
Name: 2, dtype: object

In [76]:
node = {
    "scale_name": golden_df_row_3['scale_name'],
    "children": {}
}

In [77]:
test_dict['GOLDEN_CO_SG']['scale_tree']

{'TOTAL': {'scale_name': '종합',
  'children': {'E': {'scale_name': '외향', 'children': {}}}}}

In [78]:
parent_node = find_node(test_dict['GOLDEN_CO_SG']['scale_tree'], golden_df_row_3['parent_scale_id'])
print(parent_node)

현재 depth에서 찾음
{'scale_name': '종합', 'children': {'E': {'scale_name': '외향', 'children': {}}}}


In [79]:
parent_node['children'][golden_df_row_3['scale_code']] = node
print(test_dict)

{'GOLDEN_CO_SG': {'test_name': '골든 성격유형 검사', 'essential_info': '{sex, age}', 'version': 'V 1.0', 'scale_tree': {'TOTAL': {'scale_name': '종합', 'children': {'E': {'scale_name': '외향', 'children': {}}, 'T': {'scale_name': '사고', 'children': {}}}}}}}


In [87]:
golden_df_row_4 = golden_df.iloc[3]
print(golden_df_row_4)
node = {
    "scale_name": golden_df_row_4['scale_name'],
    "children": {}
}
print(node)

test_id            GOLDEN_CO_SG
test_name            골든 성격유형 검사
essential_info       {sex, age}
scale_code                  a01
scale_name                말이 많은
parent_scale_id               E
level                         3
version                   V 1.0
Name: 4, dtype: object
{'scale_name': '말이 많은', 'children': {}}


In [83]:
test_dict['GOLDEN_CO_SG']['scale_tree']

{'TOTAL': {'scale_name': '종합',
  'children': {'E': {'scale_name': '외향', 'children': {}},
   'T': {'scale_name': '사고', 'children': {}}}}}

In [88]:
test_dict['GOLDEN_CO_SG']['scale_tree'].values()

dict_values([{'scale_name': '종합', 'children': {'E': {'scale_name': '외향', 'children': {}}, 'T': {'scale_name': '사고', 'children': {}}}}])

In [95]:
for nodo in test_dict['GOLDEN_CO_SG']['scale_tree'].values():
    print(nodo)


{'scale_name': '종합', 'children': {'E': {'scale_name': '외향', 'children': {}}, 'T': {'scale_name': '사고', 'children': {}}}}


In [94]:
"children" in nodo

True

In [99]:
nodo["children"]

{'E': {'scale_name': '외향', 'children': {}},
 'T': {'scale_name': '사고', 'children': {}}}

In [98]:
golden_df_row_4['parent_scale_id'] in nodo["children"]

True

In [100]:
nodo["children"][golden_df_row_4['parent_scale_id']]

{'scale_name': '외향', 'children': {}}

In [220]:
def build_flat_cache(scale_tree, test_id, path=None):
    cache = {}
    if path is None:
        path = []

    for code, node in scale_tree.items():
        current_path = path + [code]   # 경로 누적
        cache[(test_id, code)] = {
            "node": node,              # 노드 참조 (scale_name, children 등)
            "path": current_path       # 루트부터 현재까지 경로
        }
        # 재귀적으로 children 탐색
        if node["children"]:
            child_cache = build_flat_cache(node["children"], test_id, current_path)
            cache.update(child_cache)
    return cache

In [221]:
tree_dict = {'ADHD_CO_SG': {'test_name': '주의력 검사', 'essential_info': '{age}', 'version': 'V 1.0', 'scale_tree': {'A': {'scale_name': '주의력', 'children': {}}, 'B': {'scale_name': '과잉행동', 'children': {}}}}, 'GOLDEN_CO_SG': {'test_name': '골든 성격유형 검사', 'essential_info': '{sex, age}', 'version': 'V 1.0', 'scale_tree': {'TOTAL': {'scale_name': '종합', 'children': {'E': {'scale_name': '외향', 'children': {'a01': {'scale_name': '말이 많은', 'children': {}}, 'a02': {'scale_name': '친밀한', 'children': {}}}}, 'T': {'scale_name': '사고', 'children': {'t01': {'scale_name': '논리적', 'children': {}}, 't02': {'scale_name': '분석적', 'children': {}}}}}}}}, 'IESS_CO_SG': {'test_name': '통합 스트레스 검사', 'essential_info': '{age, position, role}', 'version': 'V 1.0', 'scale_tree': {'A': {'scale_name': '비관주의', 'children': {}}, 'B': {'scale_name': '수면문제', 'children': {}}, 'C': {'scale_name': '우울', 'children': {}}}}, 'OHSS_CO_SG': {'test_name': '직무스트레스 검사', 'essential_info': '{sex, age, position, role}', 'version': 'V 1.0', 'scale_tree': {'TOTAL': {'scale_name': '종합', 'children': {'A': {'scale_name': '스트레스', 'children': {'A01': {'scale_name': '직무부하', 'children': {}}, 'A02': {'scale_name': '업무갈등', 'children': {}}}}}}}}}
tree_dict

{'ADHD_CO_SG': {'test_name': '주의력 검사',
  'essential_info': '{age}',
  'version': 'V 1.0',
  'scale_tree': {'A': {'scale_name': '주의력', 'children': {}},
   'B': {'scale_name': '과잉행동', 'children': {}}}},
 'GOLDEN_CO_SG': {'test_name': '골든 성격유형 검사',
  'essential_info': '{sex, age}',
  'version': 'V 1.0',
  'scale_tree': {'TOTAL': {'scale_name': '종합',
    'children': {'E': {'scale_name': '외향',
      'children': {'a01': {'scale_name': '말이 많은', 'children': {}},
       'a02': {'scale_name': '친밀한', 'children': {}}}},
     'T': {'scale_name': '사고',
      'children': {'t01': {'scale_name': '논리적', 'children': {}},
       't02': {'scale_name': '분석적', 'children': {}}}}}}}},
 'IESS_CO_SG': {'test_name': '통합 스트레스 검사',
  'essential_info': '{age, position, role}',
  'version': 'V 1.0',
  'scale_tree': {'A': {'scale_name': '비관주의', 'children': {}},
   'B': {'scale_name': '수면문제', 'children': {}},
   'C': {'scale_name': '우울', 'children': {}}}},
 'OHSS_CO_SG': {'test_name': '직무스트레스 검사',
  'essential_info': '

In [222]:
flat_cache = build_flat_cache(tree_dict['GOLDEN_CO_SG']['scale_tree'], 'GOLDEN_CO_SG')
flat_cache

{('GOLDEN_CO_SG',
  'TOTAL'): {'node': {'scale_name': '종합',
   'children': {'E': {'scale_name': '외향',
     'children': {'a01': {'scale_name': '말이 많은', 'children': {}},
      'a02': {'scale_name': '친밀한', 'children': {}}}},
    'T': {'scale_name': '사고',
     'children': {'t01': {'scale_name': '논리적', 'children': {}},
      't02': {'scale_name': '분석적', 'children': {}}}}}}, 'path': ['TOTAL']},
 ('GOLDEN_CO_SG',
  'E'): {'node': {'scale_name': '외향',
   'children': {'a01': {'scale_name': '말이 많은', 'children': {}},
    'a02': {'scale_name': '친밀한', 'children': {}}}}, 'path': ['TOTAL', 'E']},
 ('GOLDEN_CO_SG', 'a01'): {'node': {'scale_name': '말이 많은', 'children': {}},
  'path': ['TOTAL', 'E', 'a01']},
 ('GOLDEN_CO_SG', 'a02'): {'node': {'scale_name': '친밀한', 'children': {}},
  'path': ['TOTAL', 'E', 'a02']},
 ('GOLDEN_CO_SG',
  'T'): {'node': {'scale_name': '사고',
   'children': {'t01': {'scale_name': '논리적', 'children': {}},
    't02': {'scale_name': '분석적', 'children': {}}}}, 'path': ['TOTAL', 'T']}

In [ ]:
# 이 데이터 원본에 선택값을 추가하려고 함
tree_dict['GOLDEN_CO_SG']['scale_tree']

{'TOTAL': {'scale_name': '종합',
  'children': {'E': {'scale_name': '외향',
    'children': {'a01': {'scale_name': '말이 많은', 'children': {}},
     'a02': {'scale_name': '친밀한', 'children': {}}}},
   'T': {'scale_name': '사고',
    'children': {'t01': {'scale_name': '논리적', 'children': {}},
     't02': {'scale_name': '분석적', 'children': {}}}}}}}

In [224]:
# flat_cache의 path를 활용해서
flat_cache[('GOLDEN_CO_SG', 'a01')]['path']

['TOTAL', 'E', 'a01']

In [231]:
target_tree_dict = tree_dict['GOLDEN_CO_SG']['scale_tree']
for scale_code in flat_cache[('GOLDEN_CO_SG', 'a01')]['path']:
    print(scale_code)
    if scale_code in target_tree_dict:
        target_tree_dict = target_tree_dict[scale_code]
    elif 'children' in target_tree_dict:
        target_tree_dict = target_tree_dict['children'][scale_code]
    else:
        raise KeyError(f"Scale code {scale_code} not found in the tree.")
    # 중간 상태 확인
    print(target_tree_dict)

# 최종 상태 확인
print(target_tree_dict)

TOTAL
{'scale_name': '종합', 'children': {'E': {'scale_name': '외향', 'children': {'a01': {'scale_name': '말이 많은', 'children': {}}, 'a02': {'scale_name': '친밀한', 'children': {}}}}, 'T': {'scale_name': '사고', 'children': {'t01': {'scale_name': '논리적', 'children': {}}, 't02': {'scale_name': '분석적', 'children': {}}}}}}
E
{'scale_name': '외향', 'children': {'a01': {'scale_name': '말이 많은', 'children': {}}, 'a02': {'scale_name': '친밀한', 'children': {}}}}
a01
{'scale_name': '말이 많은', 'children': {}}
{'scale_name': '말이 많은', 'children': {}}


In [232]:
target_tree_dict['selected'] = True
print(target_tree_dict)
print(tree_dict['GOLDEN_CO_SG']['scale_tree'])

{'scale_name': '말이 많은', 'children': {}, 'selected': True}
{'TOTAL': {'scale_name': '종합', 'children': {'E': {'scale_name': '외향', 'children': {'a01': {'scale_name': '말이 많은', 'children': {}, 'selected': True}, 'a02': {'scale_name': '친밀한', 'children': {}}}}, 'T': {'scale_name': '사고', 'children': {'t01': {'scale_name': '논리적', 'children': {}}, 't02': {'scale_name': '분석적', 'children': {}}}}}}}
